In [63]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import month
from pyspark.sql.functions import year
from pyspark.sql.functions import col
from pyspark.sql.functions import round
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import sum_distinct

spark = SparkSession.builder \
.appName("read")  \
.getOrCreate()

"""
Таблица movies:

movie_id: ID фильма
title: Название фильма
genre: Жанр фильма
release_date: Дата выхода (в формате YYYY-MM-DD)
budget: Бюджет фильма

Таблица actors:

actor_id: ID актера
name: Имя актера
birth_date: Дата рождения актера (в формате YYYY-MM-DD)
country: Страна актера
Таблица movie_actors:

movie_id: ID фильма
actor_id: ID актера
"""
"""
3. SQL запросы:

a) Найдите топ-5 жанров по количеству фильмов.
b) Найдите актера с наибольшим количеством фильмов.
c) Подсчитайте средний бюджет фильмов по жанрам.
d) Найдите фильмы, в которых снялось более одного актера из одной страны.
"""
df_movies = spark.read.option("header","true").csv("/content/movies.csv")
df_movie_actors = spark.read.option("header","true").csv("/content/movie_actors.csv")
df_actors = spark.read.option("header","true").csv("/content/actors.csv")

df_movies_types = (df_movies.withColumn("movie_id", df_movies["movie_id"].cast('int'))
                 .withColumn("release_date", df_movies["release_date"].cast('date'))
                 .withColumn("budget", df_movies["budget"].cast(DecimalType(18, 2)))
)
df_movie_actors_types = (df_movie_actors.withColumn("movie_id", df_movie_actors["movie_id"].cast('int'))
                 .withColumn("actor_id", df_movie_actors["actor_id"].cast('int'))
)
df_actors_types = (df_actors.withColumn("actor_id", df_actors["actor_id"].cast('int'))
                 .withColumn("birth_date", df_actors["birth_date"].cast('date'))
)

df1 =  (df_movies_types.join(df_movie_actors_types, ['movie_id']
                           , how="left"))
df =  (df_actors_types.join(df1, ['actor_id']
                           , how="left"))
# a
df_genre = (df1.groupBy("genre").agg(countDistinct("movie_id","genre"))
.withColumnRenamed("count(DISTINCT movie_id, genre)", "cnt_genre"))
df_genre.select("genre","cnt_genre").orderBy(col("cnt_genre").desc()).show(5)

# b
df_actors_in_movies = (df.groupBy("actor_id", "name").agg(countDistinct("movie_id","actor_id", "name"))
.withColumnRenamed("count(DISTINCT movie_id, actor_id, name)", "cnt_movie"))
df_actors_in_movies.select("name","cnt_movie").orderBy(col("cnt_movie").desc()).show(1)

# c
df_genre_budget = (df1.groupBy("genre").agg(sum_distinct("budget"), countDistinct("budget"))
.withColumnRenamed("sum(DISTINCT budget)", "sum_budget")
.withColumnRenamed("count(DISTINCT budget)", "cnt_budget")
)
df_genre_budget.select("genre", round(df_genre_budget.sum_budget / df_genre_budget.cnt_budget,2).alias("avg_budget")).orderBy(col("avg_budget").desc()).show()

# d
df_actors_in_movies = (df.filter(df.title.isNotNull()).groupBy("title", "country").agg(countDistinct("actor_id"))
.withColumnRenamed("count(DISTINCT actor_id)", "cnt_actor"))
df_max_actors_in_movies = (df_actors_in_movies.groupBy("title", "country").agg({"cnt_actor": "max"})
.withColumnRenamed("max(cnt_actor)", "max_actor").where(col('max_actor')>1))
df_max_actors_in_movies.orderBy(col("max_actor").desc()).show()

+------+---------+
| genre|cnt_genre|
+------+---------+
| Drama|        6|
|Action|        6|
|Comedy|        4|
|Horror|        2|
|Sci-Fi|        2|
+------+---------+

+--------+---------+
|    name|cnt_movie|
+--------+---------+
|Actor_24|        5|
+--------+---------+
only showing top 1 row

+------+-----------+
| genre| avg_budget|
+------+-----------+
|Horror|87281876.78|
|Sci-Fi|78097151.75|
| Drama|60760218.56|
|Comedy|52070966.22|
|Action|27492742.56|
+------+-----------+

+--------+---------+---------+
|   title|  country|max_actor|
+--------+---------+---------+
| Movie_7|    India|        2|
| Movie_3|      USA|        2|
|Movie_10|       UK|        2|
|Movie_15|    India|        2|
|Movie_18|Australia|        2|
| Movie_1|    India|        2|
| Movie_7|      USA|        2|
|Movie_10|      USA|        2|
+--------+---------+---------+

